In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

class GameNode:
    def __init__(self, stones_left, p1_stones, p2_stones, p1_points, p2_points, current_player):
        """
        stones_left: how many stones remain on the table
        p1_stones: how many stones the human player currently holds
        p2_stones: how many stones the computer currently holds
        p1_points: current points for the human player
        p2_points: current points for the computer
        current_player: 1 for human's turn, 2 for computer's turn
        """
        self.stones_left = stones_left
        self.p1_stones = p1_stones
        self.p2_stones = p2_stones
        self.p1_points = p1_points
        self.p2_points = p2_points
        self.current_player = current_player

    def copy(self):
        return GameNode(
            self.stones_left,
            self.p1_stones,
            self.p2_stones,
            self.p1_points,
            self.p2_points,
            self.current_player,
        )

    def get_children(self):
        """
        Generate possible next states by taking 2 or 3 stones (if possible).
        Returns a list of GameNode states for the next move.
        """
        children = []
        for take in [2, 3]:
            if self.stones_left >= take:
                child = self.copy()
                if child.current_player == 1:
                    child.p1_stones += take
                else:
                    child.p2_stones += take
                child.stones_left -= take

                # Check parity for points
                if child.stones_left % 2 == 0:
                    if child.current_player == 1:
                        child.p1_points += 2
                    else:
                        child.p2_points += 2
                else:
                    if child.current_player == 1:
                        child.p1_points -= 2
                    else:
                        child.p2_points -= 2

                # Switch turn
                child.current_player = 1 if child.current_player == 2 else 2
                children.append(child)
        return children

    def is_terminal(self):
        """
        The game is "normally" over when stones_left == 0.
        However, we may also end the game if no one can move (e.g., 1 stone left).
        For normal end, just check if stones_left == 0.
        """
        return self.stones_left == 0

    def get_winner(self):
        """
        Determine the game's outcome: 
            1 -> human wins
            2 -> computer wins
            0 -> tie
        At the end, each player's points are increased by the number of stones they hold.
        """
        final_p1 = self.p1_points + self.p1_stones
        final_p2 = self.p2_points + self.p2_stones
        if final_p1 > final_p2:
            return 1
        elif final_p2 > final_p1:
            return 2
        else:
            return 0

def evaluate(node):
    """
    Heuristic or terminal evaluation:
    - If node is terminal, +inf if computer wins, -inf if human wins, 0 if tie.
    - Otherwise, difference in (computer's total) - (human's total).
    """
    if node.is_terminal():
        winner = node.get_winner()
        if winner == 2:
            return float('inf')
        elif winner == 1:
            return float('-inf')
        else:
            return 0  # tie
    comp_total = node.p2_points + node.p2_stones
    hum_total = node.p1_points + node.p1_stones
    return comp_total - hum_total

class SearchAlgorithms:
    def __init__(self, max_depth=4):
        self.max_depth = max_depth
        self.nodes_visited = 0

    def minimax(self, node, depth, maximizing_player):
        self.nodes_visited += 1

        if depth == 0 or node.is_terminal():
            return evaluate(node), None

        children = node.get_children()
        if not children:  # no moves => treat as terminal from the search perspective
            return evaluate(node), None

        if maximizing_player:
            best_val = float('-inf')
            best_child = None
            for child in children:
                val, _ = self.minimax(child, depth - 1, False)
                if val > best_val:
                    best_val = val
                    best_child = child
            return best_val, best_child
        else:
            best_val = float('inf')
            best_child = None
            for child in children:
                val, _ = self.minimax(child, depth - 1, True)
                if val < best_val:
                    best_val = val
                    best_child = child
            return best_val, best_child

    def alphabeta(self, node, depth, alpha, beta, maximizing_player):
        self.nodes_visited += 1

        if depth == 0 or node.is_terminal():
            return evaluate(node), None

        children = node.get_children()
        if not children:
            return evaluate(node), None

        if maximizing_player:
            best_val = float('-inf')
            best_child = None
            for child in children:
                val, _ = self.alphabeta(child, depth - 1, alpha, beta, False)
                if val > best_val:
                    best_val = val
                    best_child = child
                alpha = max(alpha, best_val)
                if alpha >= beta:
                    break
            return best_val, best_child
        else:
            best_val = float('inf')
            best_child = None
            for child in children:
                val, _ = self.alphabeta(child, depth - 1, alpha, beta, True)
                if val < best_val:
                    best_val = val
                    best_child = child
                beta = min(beta, best_val)
                if beta <= alpha:
                    break
            return best_val, best_child

class StonesGameGUI:
    def __init__(self):
        # Widgets
        self.stones_slider = widgets.IntSlider(value=50, min=50, max=70, step=1, description="Stones:")
        self.starter_dropdown = widgets.Dropdown(options=["Human", "Computer"], value="Human", description="Who starts?")
        self.algorithm_dropdown = widgets.Dropdown(options=["Minimax", "Alpha-Beta"], value="Minimax", description="Algorithm:")
        self.start_button = widgets.Button(description="Start New Game")

        self.take2_button = widgets.Button(description="Take 2 stones")
        self.take3_button = widgets.Button(description="Take 3 stones")

        self.output_area = widgets.Output()

        # Game state
        self.current_node = None
        self.game_over = False
        self.ai_search = SearchAlgorithms(max_depth=6)
        self.ai_algorithm = None
        
        # Experiment tracking (optional)
        self.experiments_data = {
            "minimax": {"wins_computer": 0, "wins_human": 0, "nodes_visited": 0, "move_count": 0, "total_time": 0.0},
            "alphabeta": {"wins_computer": 0, "wins_human": 0, "nodes_visited": 0, "move_count": 0, "total_time": 0.0}
        }

        # Event bindings
        self.start_button.on_click(self.on_start_game)
        self.take2_button.on_click(self.on_take2)
        self.take3_button.on_click(self.on_take3)

        # Layout
        self.ui = widgets.VBox([
            widgets.HBox([self.stones_slider, self.starter_dropdown, self.algorithm_dropdown, self.start_button]),
            widgets.HBox([self.take2_button, self.take3_button]),
            self.output_area
        ])
        display(self.ui)

    def on_start_game(self, _):
        with self.output_area:
            clear_output()
            print("Starting a new game...")

        # Reset
        stones = self.stones_slider.value
        self.current_node = GameNode(stones, 0, 0, 0, 0, 1 if self.starter_dropdown.value == "Human" else 2)
        self.game_over = False
        
        algo_choice = self.algorithm_dropdown.value
        self.ai_algorithm = "minimax" if algo_choice == "Minimax" else "alphabeta"

        self._print_game_state()
        if self.current_node.current_player == 2:
            self.computer_move()

    def on_take2(self, _):
        if self.game_over or self.current_node.current_player != 1:
            return
        self.make_move(2)

    def on_take3(self, _):
        if self.game_over or self.current_node.current_player != 1:
            return
        self.make_move(3)

    def make_move(self, take_amount):
        """
        Called when the human tries to take 2 or 3 stones.
        If the move is not possible (e.g., only 1 stone left), we skip.
        If the next player also cannot move, the game ends.
        Otherwise, if the move is valid, we apply it and then let the computer move.
        """
        if self.game_over:
            return
        
        node = self.current_node
        # Check if user can actually take 2 or 3 stones
        if node.stones_left < take_amount:
            with self.output_area:
                print("No valid move for the human (can't take 2 or 3). Skipping turn...")
            # Skip to computer
            node.current_player = 2
            if not node.get_children():  # If the computer also has no moves, end game
                self._end_game()
            else:
                self._print_game_state()
                self.computer_move()
            return

        # If move is valid, apply it
        self.apply_move(take_amount)
        
        # If game not over, let the computer move
        if not self.game_over and self.current_node.current_player == 2:
            self.computer_move()

    def computer_move(self):
        """
        AI turn. If computer has no moves, it skips. 
        If the human also then has no moves, game ends.
        Otherwise AI picks its best move.
        """
        if self.game_over or self.current_node.current_player != 2:
            return

        # Check if computer can move
        if not self.current_node.get_children():
            with self.output_area:
                print("No valid move for the computer. Skipping turn...")
            # Skip to human
            self.current_node.current_player = 1
            if not self.current_node.get_children():  # If human also cannot move, end game
                self._end_game()
            else:
                self._print_game_state()
            return
        
        # Normal AI logic
        start_time = time.time()
        self.ai_search.nodes_visited = 0
        
        if self.ai_algorithm == "minimax":
            _, best_child = self.ai_search.minimax(self.current_node, self.ai_search.max_depth, True)
        else:
            _, best_child = self.ai_search.alphabeta(self.current_node, self.ai_search.max_depth, float('-inf'), float('inf'), True)
        
        visited = self.ai_search.nodes_visited
        elapsed = time.time() - start_time
        
        data_key = "minimax" if self.ai_algorithm == "minimax" else "alphabeta"
        self.experiments_data[data_key]["nodes_visited"] += visited
        self.experiments_data[data_key]["move_count"] += 1
        self.experiments_data[data_key]["total_time"] += elapsed

        if not best_child:
            # No moves found, end game
            self._end_game()
            return

        self.current_node = best_child
        if self.current_node.is_terminal():
            self._end_game()
        else:
            self._print_game_state()

    def apply_move(self, take_amount):
        """
        Apply a move (either human or computer) by taking `take_amount` stones, 
        and adjust points by parity.
        """
        node = self.current_node

        # The current player takes the stones
        if node.current_player == 1:
            node.p1_stones += take_amount
        else:
            node.p2_stones += take_amount
        node.stones_left -= take_amount

        # Adjust points: +2 if the resulting stones_left is even, -2 if odd
        if node.stones_left % 2 == 0:
            if node.current_player == 1:
                node.p1_points += 2
            else:
                node.p2_points += 2
        else:
            if node.current_player == 1:
                node.p1_points -= 2
            else:
                node.p2_points -= 2

        # Switch turn
        node.current_player = 2 if node.current_player == 1 else 1

        # If the move caused the game to have 0 stones, end
        if node.is_terminal():
            self._end_game()
        else:
            self._print_game_state()

    def _end_game(self):
        """Finalize the game and announce the result."""
        self.game_over = True
        winner = self.current_node.get_winner()
        final_p1 = self.current_node.p1_points + self.current_node.p1_stones
        final_p2 = self.current_node.p2_points + self.current_node.p2_stones
        
        with self.output_area:
            self._print_game_state()
            if winner == 1:
                print(f"Game Over! Human wins: {final_p1} vs {final_p2}")
            elif winner == 2:
                print(f"Game Over! Computer wins: {final_p2} vs {final_p1}")
            else:
                print(f"Game Over! It's a tie: {final_p1} vs {final_p2}")

        # Record result in experiments_data
        data_key = "minimax" if self.ai_algorithm == "minimax" else "alphabeta"
        if winner == 1:
            self.experiments_data[data_key]["wins_human"] += 1
        elif winner == 2:
            self.experiments_data[data_key]["wins_computer"] += 1

    def _print_game_state(self):
        with self.output_area:
            clear_output(wait=True)
            print("Current Game State:")
            print(f"  Stones left on table: {self.current_node.stones_left}")
            print(f"  Human: stones={self.current_node.p1_stones}, points={self.current_node.p1_points}")
            print(f"  Computer: stones={self.current_node.p2_stones}, points={self.current_node.p2_points}")
            current = "Human" if self.current_node.current_player == 1 else "Computer"
            print(f"  Next turn: {current}")
            print("----------------------------------")

# Instantiate the game GUI
game_gui = StonesGameGUI()

# Game Over! - ja cilveks uzvar
# Punktu skaitisana/displayot points vajag - 50 vs 38 exmp
# Neizskirts gadijums - 


TraitError: The 'value' trait of an IntSlider instance expected an int, not the NoneType None.

TraitError: The 'value' trait of an IntSlider instance expected an int, not the NoneType None.

TraitError: The 'value' trait of an IntSlider instance expected an int, not the NoneType None.

AttributeError: 'SearchAlgorithms' object has no attribute '_end_game'

TraitError: The 'value' trait of an IntSlider instance expected an int, not the NoneType None.

AttributeError: 'SearchAlgorithms' object has no attribute '_end_game'

AttributeError: 'SearchAlgorithms' object has no attribute '_end_game'

AttributeError: 'SearchAlgorithms' object has no attribute '_end_game'

AttributeError: 'SearchAlgorithms' object has no attribute '_end_game'